# 과제
### 2022235027 민현기


- 답변의 제출은 반드시 "학번.pdf"로 제출하고, 추가적으로 분석에 활용된 python code를 원본형식으로 제출하라.
    - (코드 파일에 # 이나 Jupyternote book 기능을 이용해 설명만 추가하는 형식의 제출은 정식 과제 점수로 인정하지 않음.)

In [1]:
import requests
from urllib import parse

In [2]:
def matzip_api(coordinate):
    address = 'https://www.daegufood.go.kr/kor/api/tasty.html?mode=json&addr='
    coordinate = parse.quote(coordinate)
    response = requests.get(address+coordinate)
    
    return response.json(strict=False)

In [3]:
location = ['중구', '동구', '서구', '남구', '북구', '수성구', '달서구', '달성군']

In [4]:
daegu_matzip = {i:matzip_api(i) for i in location}

### 1. 대구의 7개 구(중구, 동구, 서구, 남구, 북구, 수성구, 달서구)과 1개 군(달성군)의 맛집은 각각 몇 개인가?

In [5]:
for i in daegu_matzip:
    print(f'{i}: {daegu_matzip[i]["total"]} 개')

중구: 181 개
동구: 137 개
서구: 63 개
남구: 57 개
북구: 95 개
수성구: 131 개
달서구: 168 개
달성군: 93 개


### 2. 오픈 api에 포함된 "설명"(SMPL_DESC)을 분석하여, 맛집 설명에서 가장 유의미하게 많이 등장하는 5개 단어를 제시하라.

In [6]:
import numpy as np
import pandas as pd
import re
from konlpy.tag import Okt
import gensim
from gensim import corpora

In [7]:
def make_df(location):
    df = []
    for i in location:
        local_df = pd.DataFrame(daegu_matzip[i]['data'])
        local_df['LOCATION'] = i
        df.append(local_df)
        
    return pd.concat(df).reset_index(drop=True)

In [8]:
df = make_df(location)

In [9]:
class CustomLDA:
    def __init__(self, pandas_text_column, stop_words, num_topics=1, num_words=5):
        clean = re.compile("[^ㄱ-힣0-9 %]")
        self.texts = pandas_text_column.apply(lambda x: clean.sub('', str(x)))
        self.tagger = Okt()
        self.stop_words = stop_words
        self.num_topics = num_topics
        self.num_words = num_words
        
    def nouns_tokenizer(self, text):
        word_token = self.tagger.nouns(text)
        self.result = [word for word in word_token if word not in self.stop_words]
        return self.result
    
    def LDA(self):
        nouns_tokenizer = self.nouns_tokenizer
        texts = self.texts.apply(nouns_tokenizer)
        dictionary = corpora.Dictionary(texts)
        corpus = [dictionary.doc2bow(text) for text in texts]
        NUM_TOPICS = self.num_topics
        ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
        topics = ldamodel.print_topics(num_words=self.num_words)
        return topics

- 처음엔 불용어를 설정하지 않고, 결과를 돌려보며 리스트에 나오는 불용어를 하나씩 제거하며 최종적으로 5개의 단어를 선정함

In [10]:
stop_words = ['수', '요리', '곳', '등', '음식점', '한우', '사용', '음식']
CustomLDA(pandas_text_column=df['SMPL_DESC'], stop_words=stop_words, num_topics=1).LDA()

[(0, '0.043*"전문점" + 0.012*"맛" + 0.010*"위치" + 0.008*"전통" + 0.008*"직접"')]

### 3. 오픈 api에 포함된 "메뉴"(MNU)를 분석하여, 대구 맛집들이 제공하는 가장 흔한 메뉴 3개를 제시하라. 그리고 이들의 평균 가격 또한 함께 제시하라.

In [11]:
def make_menu_df(text):
    text = text.strip('\r\n').split('\r\n')
    
    pattern_price = re.compile('[0-9,[0-9]+(?=원)')
    pattern_menu = re.compile('^(.*?)(?= \d+,)')
    
    menu = pd.Series([pattern_menu.findall(i) for i in text]).apply(lambda x: ''.join(x) if x else '없음')
    # menu = pd.Series(text).apply(CustomLDA(pd.Series(text[0]), stop_words).nouns_tokenizer).apply(lambda x: ''.join(x))
    price = pd.Series([pattern_price.findall(i) for i in text]).apply(lambda x: int(x[0].replace(',', '')) if x else np.nan)
    
    return pd.DataFrame(zip(menu, price), columns=['품목', '가격'])

In [12]:
def menu_concat(df):
    concat_list_df = [make_menu_df(df['MNU'][i]) for i in range(len(df['MNU']))]
    return pd.concat(concat_list_df).reset_index(drop=True), concat_list_df

In [13]:
menu_df, store_df = menu_concat(df)

In [14]:
stop_words_menu = ['']
CustomLDA(menu_df['품목'], stop_words=stop_words_menu, num_topics=1, num_words=3).LDA()

[(0, '0.016*"한우" + 0.016*"불고기" + 0.013*"정식"')]

In [15]:
menu_df[menu_df['품목'].str.contains('한우')]

,품목,가격
14,디너 한우 채끝등심 코스,75000.0
198,한우 불고기정식,11000.0
199,한우 갈비탕,13000.0
200,한우 갈비찜정식,19000.0
203,한우한마리(300g),45000.0
...,...,...
6351,한우갈비살(100g),17000.0
6528,한우모듬세트(1인분 100g),5800.0
6531,한우 토시살(1인분 100g),13500.0
6532,한우 안창살(1인분 100g),13500.0


In [16]:
menu_df[menu_df['품목'].str.contains('불고기')]

,품목,가격
70,닭불고기,17000.0
126,불고기퀘사디아,14900.0
198,한우 불고기정식,11000.0
222,오리불고기,25000.0
276,은복불고기(1인분),13000.0
...,...,...
6554,버섯불고기(1인분),11000.0
6580,청둥오리불고기,40000.0
6596,메기불고기,40000.0
6608,흑염소 돌판불고기(2인분),40000.0


In [17]:
menu_df[menu_df['품목'].str.contains('정식')]

,품목,가격
40,정담정식,13000.0
139,정식,6000.0
198,한우 불고기정식,11000.0
200,한우 갈비찜정식,19000.0
281,특선연요리정식(1인),20000.0
...,...,...
6341,돈까스정식,11900.0
6344,어린이정식,6900.0
6494,정강희특정식,15000.0
6515,고등어정식(2인이상),7000.0


In [18]:
select_menu = ['한우', '불고기', '정식']
for menu in select_menu:
    price = menu_df[menu_df['품목'].str.contains(menu)]['가격'].mean()
    print(f"{menu} 평균 가격: {round(price)} 원")

한우 평균 가격: 22697 원
불고기 평균 가격: 17207 원
정식 평균 가격: 17028 원


### 4. 본인이 제시한 2번, 3번 결과에 부합하는 맛집을 최종적으로 최대 3곳 선정하여 제시하라. (논리적 근거를 명확히 밝히라.)

- 맛집을 선정하기 위한 가정과 적용방식
    - 가정 
        - 음식 가격이 비싼만큼 맛이 뛰어나 사람들이 좋아하고, 가격이 싼 만큼 가성비가 뛰어나 사람들이 좋아한다.
        - 3번 문제의 결과는 사람들의 선호도가 반영된 결과이다(사람들은 `한우, 불고기, 정식`을 특히 선호한다).
        - 식당 메뉴가 다양할수록 사람들이 좋아할만한 음식이 존재할 확률이 높다.
    - 가정에 대한 가중치 적용
        - 식당 메뉴 평균 가격이 전체 가게 평균에서 높거나 낮은 경우 가중치 부여
        - 3가지 음식(한우, 불고기, 정식)의 요소가 포함되는 메뉴를 가진 식당의 경우 각각 가중치 부여
            - 특히, 맛있는(비싼)음식이 더 가중치를 줄 수 있도록, 3가지 음식 가격의 합계에서 각 음식의 가격의 비율만큼의 가중치를 추가(한 메뉴에 중복된 경우 가장 큰 것을 적용)
    - 계산식
        - $$|log{(0.1+\frac{개별식당카테고리별가격점수^*}{카테고리별전체식당평균가격}})|$$
            - *개별식당카테고리별가격점수: 식당 개별 메뉴에 대한 가중치를 계산 후의 평균 가격
                1) 전체식당평균가격보다 높은경우의 개별음식 가중치: 한우:40%, 불고기:30%, 정식:30% 가격 추가
                2) 전체식당평균가격보다 낮은경우의 개별음식 가중치: 한우:30%, 불고기:20%, 정식:20% 가격 할인(log함수가 값이 작을수록 급격히 커지는 것을 고려하여 조정)

In [19]:
price_all = [store_df[i]['가격'].mean() for i in range(len(store_df))]
price_all_mean = np.nanmean(price_all)

In [20]:
store_df[0]

,품목,가격
0,복어매운탕,9000
1,복어지리,9000
2,복어수육,45000
3,모듬회,20000
4,껍질무침,15000


In [21]:
df['FD_CS'].unique()

array(['한식', '양식', '세계요리', '일식', '중식', '디저트/베이커리', '전통차/커피전문점', '특별한 술집',
       '퓨전/뷔페'], dtype=object)

In [22]:
### 카테고리별전체식당평균가격
category_dict = {'한식': [],
                '양식': [],
                '세계요리': [],
                '일식': [],
                '중식': [],
                '디저트/베이커리': [],
                '전통차/커피전문점': [],
                '특별한 술집': [],
                '퓨전/뷔페': []}

for i in range(len(store_df)):
    category = df['FD_CS'][i]
    category_dict[category].append(store_df[i]['가격'].mean())

In [23]:
category_dict_mean_price = {}

for i in category_dict:
    category_dict_mean_price[i] = np.nanmean(category_dict[i])

In [24]:
category_dict_mean_price

{'한식': 19841.000682209422,
 '양식': 23945.30892549185,
 '세계요리': 17996.95079718436,
 '일식': 37225.54934381857,
 '중식': 19790.487213403878,
 '디저트/베이커리': 5355.0066000066,
 '전통차/커피전문점': 6808.876137712702,
 '특별한 술집': 13831.349206349207,
 '퓨전/뷔페': 36110.416666666664}

In [25]:
### 카테고리별개별식당가격점수

adjust_store_score = []

for i in range(len(store_df)):
    store_mean = store_df[i]['가격'].mean()
    adjust_price = []
    category = df['FD_CS'][i]
    
    for menu, price in zip(store_df[i]['품목'], store_df[i]['가격']):
        if store_mean >= category_dict_mean_price[category]:
            if '한우' in menu:
                adjust_price.append(price + price*0.4)
            elif ('불고기' or '정식') in menu:
                adjust_price.append(price + price*0.3)
            else:
                adjust_price.append(price)
        else:
            if '한우' in menu:
                adjust_price.append(price - price*0.2)
            elif ('불고기' or '정식') in menu:
                adjust_price.append(price - price*0.15)
            else:
                adjust_price.append(price)
                
    adjust_price = np.nanmean(adjust_price)
    adjust_store_score.append([category, adjust_price])

C:\Users\hyeon\AppData\Local\Temp\ipykernel_7564\358722784.py:26: RuntimeWarning: Mean of empty slice
  adjust_price = np.nanmean(adjust_price)


In [26]:
rank_df = pd.DataFrame(adjust_store_score, columns=['category', 'score'])

In [27]:
def calc_score(score, category):
    return abs(np.log((score/category_dict_mean_price[category])+0.1))

In [28]:
rank_df['rank_score'] = [calc_score(rank_df['score'][i], rank_df['category'][i]) for i in range(len(rank_df))]

In [29]:
rank_df.sort_values(by=['rank_score'], ascending=False)[:5]

,category,score,rank_score
225,퓨전/뷔페,2000.000000,1.861845
661,세계요리,90420.000000,1.633971
623,한식,2083.333333,1.584738
682,한식,91666.666667,1.551822
491,한식,89600.000000,1.529507


In [30]:
print(df.iloc[661, :])
print(df.iloc[661, :]['MNU'])

cnt                                                          129
OPENDATA_ID                                                  147
GNG_CS                                      대구광역시 수성구 범어동 143-19
FD_CS                                                       세계요리
BZ_NM                                                       아트리움
TLNO                                                053-754-3111
MBZ_HR                                             11:00 - 23:00
SEAT_CNT                                                    120석
PKPL                                                         50대
HP                                        www.atriumkorea.co.kr/
PSB_FRN                                          일본어 기타 (인도, 네팔)
BKN_YN                                                        가능
INFN_FCL                                                     불가능
BRFT_YN                                                      불가능
DSSRT_YN                                                      가능
MNU            (런치) 아트리움런

In [31]:
print(df.iloc[623, :])
print(df.iloc[623, :]['MNU'])

cnt                                                           91
OPENDATA_ID                                                  505
GNG_CS                                    대구광역시 수성구 수성동4가 1120-2
FD_CS                                                         한식
BZ_NM                                                   윤옥연할매떡볶이
TLNO                                                053-756-7597
MBZ_HR                                             10:00 - 22:00
SEAT_CNT                                                     32석
PKPL                                                    4대외 노상주차
HP                                                            없음
PSB_FRN                                           영어 기타 (인도, 네팔)
BKN_YN                                            전화주문 가능,자리예약불가
INFN_FCL                                                     불가능
BRFT_YN                                                      불가능
DSSRT_YN                                                     불가능
MNU            떡볶이 1,000원

In [32]:
print(df.iloc[682, :])
print(df.iloc[682, :]['MNU'])

cnt                                                           19
OPENDATA_ID                                                 1649
GNG_CS                                      대구광역시 달서구 월성동 1317-1
FD_CS                                                         한식
BZ_NM                                                    신기동대게나라
TLNO                                                053-628-2277
MBZ_HR                                             11:30 ~ 23:30
SEAT_CNT                                                 96석(룸3)
PKPL                                                         50대
HP                                           pf.kakao.com/_kpSnT
PSB_FRN                                           중국어 &lt;BR&gt;
BKN_YN                                                        가능
INFN_FCL                                                     불가능
BRFT_YN                                                      불가능
DSSRT_YN                                                      가능
MNU            대게(1kg) 96